# Long Term Investment Strategy Analysis

In [2]:
pip install yfinance

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/e6/b3/388ab967a387cc92926f70e97688dd9a7189b29a0773db815ffc5289e2b5/yfinance-0.2.31-py2.py3-none-any.whl.metadata
  Using cached yfinance-0.2.31-py2.py3-none-any.whl.metadata (11 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached frozendict-2.3.8-py311-none-any.whl (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 6.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached yfinance-0.2.31-py2.py3-none-any.whl (65 kB)
  Created wheel for peewee: filename=peewee-3.17.0-cp311-cp311-macosx_10_9_x86_64.whl size=248179 sha256=7fa6cf362189ab872885c6b2fd136c4805f1c0462dd99dcc13ecd481096afb86
  Stored in directory: /Users/annatoaze/Library/Caches/pip/wheels/02/20/23/74a10d0cd31f5d41c19b9

In [3]:
# initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

import os
import requests

import yfinance as yf
import pytz

# Load and clean the data

In [34]:
#### World Stock prices

# Read the CSV file into a DataFrame and select only US stocks 
us_stock_prices_df = pd.read_csv("/Users/annatoaze/Downloads/World-Stock-Prices-Dataset.csv", parse_dates=["Date"])
us_stock_prices_df = us_stock_prices_df[us_stock_prices_df['Country'] == 'usa']

# Drop the time value in the Date column
us_stock_prices_df['Date'] = pd.to_datetime(us_stock_prices_df['Date'], utc=True)
us_stock_prices_df['Date'] = us_stock_prices_df['Date'].dt.normalize()
us_stock_prices_df['Date'] = us_stock_prices_df['Date'].dt.strftime('%Y-%m-%d')

# Drop rows with null values
us_stock_prices_df.dropna(inplace=True)

# Drop 'Dividends' and 'Stock Splits' columns as they are empty
us_stock_prices_df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)


#### S&P 500 - ^GSPC

# Define the ticker symbol
ticker = "^GSPC"

# Set the date range for historical data
start_date = "2003-10-01"
end_date = "2023-10-01"

# Fetch historical data from Yahoo Finance
SP_df = yf.download(ticker, start=start_date, end=end_date)

#Reset the index
SP_df.reset_index(inplace=True)

# Create a column called ticker
SP_df['Ticker'] = 'S&P 500'

[*********************100%%**********************]  1 of 1 completed


In [5]:
# Combine the two DataFrames into one
columns_to_concat = ['Date', 'Close', 'Open', 'High', 'Low', 'Ticker', 'Volume']

# Select the specified columns from each DataFrame
us_stock_prices_subset = us_stock_prices_df[columns_to_concat]
SP_subset = SP_df[columns_to_concat]

# Concatenate the selected columns
combined_df = pd.concat([us_stock_prices_subset, SP_subset], axis=0)

# Sort the index to ensure it's in date order
combined_df.sort_index(inplace=True)

# Ensure 'Date' column is in datetime format
combined_df['Date'] = pd.to_datetime(combined_df['Date'])

# Reformat the 'Date' column to 'YYYY-MM-DD'
combined_df['Date'] = combined_df['Date'].dt.strftime('%Y-%m-%d')

combined_df

,Date,Close,Open,High,Low,Ticker,Volume
0,2023-09-20,4.670000,4.840000,4.910000,4.630000,PTON,7.441900e+06
0,2003-10-01,1018.219971,995.969971,1018.219971,995.969971,S&P 500,1.566300e+09
1,2003-10-02,1020.239990,1018.219971,1021.869995,1013.380005,S&P 500,1.269300e+09
1,2023-09-20,386.299988,397.049988,397.989990,386.119995,NFLX,3.866600e+06
2,2023-09-20,563.830017,564.349976,569.219971,562.659973,COST,1.311500e+06
...,...,...,...,...,...,...,...
279748,2023-08-29,18.320000,18.719999,18.770000,18.020000,FL,5.949600e+06
279749,2023-08-30,18.549999,18.180000,18.650000,17.879999,FL,5.829500e+06
279750,2023-08-31,19.620001,18.620001,19.850000,18.469999,FL,6.316100e+06
279751,2023-09-01,19.870001,19.660000,20.139999,19.400000,FL,3.982400e+06


In [37]:
combined_df=combined_df[combined_df['Ticker'].isin(risk_tickers)]

NameError: name 'risk_tickers' is not defined

# SMA

In [6]:
# Select only columns 'Ticker' and 'Close' from the existing DataFrame
closing_prices_df = combined_df[['Date', 'Ticker', 'Close']]

# Set 'Date' column as the index
closing_prices_df.set_index('Date', inplace=True)

# Convert the index to datetime and extract the date component
closing_prices_df.index = pd.to_datetime(closing_prices_df.index, format='%Y-%m-%d')
closing_prices_df.index = closing_prices_df.index.date
closing_prices_df

,Ticker,Close
2023-09-20,PTON,4.670000
2003-10-01,S&P 500,1018.219971
2003-10-02,S&P 500,1020.239990
2023-09-20,NFLX,386.299988
2023-09-20,COST,563.830017
...,...,...
2023-08-29,FL,18.320000
2023-08-30,FL,18.549999
2023-08-31,FL,19.620001
2023-09-01,FL,19.870001


In [7]:
# Define the SMA window (5 years)
sma_window = 1260

# Create a new DataFrame to store the latest SMAs for each ticker
sma_summary_df = pd.DataFrame(columns=['Ticker', f'SMA_{sma_window}'])

# Loop through each unique ticker in your DataFrame
for ticker in closing_prices_df['Ticker'].unique():
    # Filter the DataFrame to get data for the current ticker and create a copy
    ticker_data = closing_prices_df[closing_prices_df['Ticker'] == ticker].copy()
    
    # Calculate the SMA for the ticker using the rolling mean and assign it using .loc
    ticker_data.loc[:, f'SMA_{sma_window}'] = ticker_data['Close'].rolling(window=sma_window).mean()
    
    # Get the latest SMA value for the ticker
    latest_sma = ticker_data.iloc[-1][f'SMA_{sma_window}']
    
    # Append the ticker and its latest SMA value to sma_summary_df
    sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)

# Sort sma_summary_df by SMA values in descending order
sma_summary_df.sort_values(by=f'SMA_{sma_window}', ascending=False, inplace=True)

# Select the top 10 tickers with the best SMA values
top_10_sma2520_tickers = sma_summary_df.head(10)

# Display the top 10 tickers with the best SMA values
print("Top 10 tickers with the best 5 year SMA values:")
print(top_10_sma2520_tickers)



/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2691124789.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2691124789.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2691124789.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/

Top 10 tickers with the best 5 year SMA values:
     Ticker     SMA_1260
1   S&P 500  3666.196245
23      CMG  1251.970469
30     ADBE   402.642230
2      NFLX   394.406190
3      COST   378.611199
40       MA   312.784917
39       HD   256.650681
14     MSFT   216.181360
38      MCD   214.881039
45      FDX   202.055938


/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2691124789.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2691124789.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2691124789.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/

In [8]:
# Define the SMA window (10 years)
sma_window = 2520

# Create a new DataFrame to store the latest SMAs for each ticker
sma_summary_df = pd.DataFrame(columns=['Ticker', f'SMA_{sma_window}'])

# Loop through each unique ticker in your DataFrame
for ticker in closing_prices_df['Ticker'].unique():
    # Filter the DataFrame to get data for the current ticker and create a copy
    ticker_data = closing_prices_df[closing_prices_df['Ticker'] == ticker].copy()
    
    # Calculate the SMA for the ticker using the rolling mean and assign it using .loc
    ticker_data.loc[:, f'SMA_{sma_window}'] = ticker_data['Close'].rolling(window=sma_window).mean()
    
    # Get the latest SMA value for the ticker
    latest_sma = ticker_data.iloc[-1][f'SMA_{sma_window}']
    
    # Append the ticker and its latest SMA value to sma_summary_df
    sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)

# Sort sma_summary_df by SMA values in descending order
sma_summary_df.sort_values(by=f'SMA_{sma_window}', ascending=False, inplace=True)

# Select the top 50 tickers with the best SMA values
top_10_sma5040_tickers = sma_summary_df.head(10)

# Display the top 50 tickers with the best SMA values
print("Top 10 tickers with the best 10 year SMA values:")
print(top_10_sma5040_tickers)


/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4088472809.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4088472809.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4088472809.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/

Top 10 tickers with the best 10 year SMA values:
     Ticker     SMA_2520
1   S&P 500  2936.420428
23      CMG   875.005504
2      NFLX   261.864115
30     ADBE   257.315695
3      COST   256.641491
40       MA   208.336652
39       HD   182.234892
45      FDX   181.979656
38      MCD   158.026850
46      MMM   138.396439


/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4088472809.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4088472809.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4088472809.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/

In [9]:
# Define the SMA window (15 years)
sma_window = 3780

# Create a new DataFrame to store the latest SMAs for each ticker
sma_summary_df = pd.DataFrame(columns=['Ticker', f'SMA_{sma_window}'])

# Loop through each unique ticker in your DataFrame
for ticker in closing_prices_df['Ticker'].unique():
    # Filter the DataFrame to get data for the current ticker and create a copy
    ticker_data = closing_prices_df[closing_prices_df['Ticker'] == ticker].copy()
    
    # Calculate the SMA for the ticker using the rolling mean and assign it using .loc
    ticker_data.loc[:, f'SMA_{sma_window}'] = ticker_data['Close'].rolling(window=sma_window).mean()
    
    # Get the latest SMA value for the ticker
    latest_sma = ticker_data.iloc[-1][f'SMA_{sma_window}']
    
    # Append the ticker and its latest SMA value to sma_summary_df
    sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)

# Sort sma_summary_df by SMA values in descending order
sma_summary_df.sort_values(by=f'SMA_{sma_window}', ascending=False, inplace=True)

# Select the top 10 tickers with the best SMA values
top_10_sma3780_tickers = sma_summary_df.head(10)

# Display the top 10 tickers with the best SMA values
print("Top 10 tickers with the best 15 year SMA values:")
print(top_10_sma3780_tickers)

/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/3540790835.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/3540790835.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/3540790835.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/

Top 10 tickers with the best 15 year SMA values:
     Ticker     SMA_3780
1   S&P 500  2367.846642
23      CMG   657.915900
3      COST   190.005765
30     ADBE   182.276699
2      NFLX   180.101784
40       MA   148.678400
45      FDX   145.670383
39       HD   131.670804
38      MCD   123.826823
46      MMM   111.299215


/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/3540790835.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/3540790835.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/3540790835.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, f'SMA_{sma_window}': latest_sma}, ignore_index=True)
/var/folders/

In [10]:
### Combine all the timeframes into a single DataFrame showing the  5, 10, and 15 year SMAs

# Define the SMA windows
sma_windows = [ 1260, 2520, 3780]  # Corresponding to 5, 10, and 15 years

# Create a new DataFrame to store the SMA results for each ticker
sma_summary_df = pd.DataFrame(columns=['Ticker'] + [f'SMA_{window}' for window in sma_windows])

# Loop through each unique ticker in your DataFrame
for ticker in closing_prices_df['Ticker'].unique():
    # Filter the DataFrame to get data for the current ticker and create a copy
    ticker_data = closing_prices_df[closing_prices_df['Ticker'] == ticker].copy()
    
    # Calculate the SMAs for the ticker using the rolling mean and assign them using .loc
    for window in sma_windows:
        column_name = f'SMA_{window}'
        ticker_data.loc[:, column_name] = ticker_data['Close'].rolling(window=window).mean()
    
    # Get the latest SMA values for the ticker
    latest_sma_values = [ticker_data.iloc[-1][f'SMA_{window}'] for window in sma_windows]
    
    # Append the ticker and its latest SMA values to sma_summary_df
    sma_summary_df = sma_summary_df.append({'Ticker': ticker, **{f'SMA_{window}': value for window, value in zip(sma_windows, latest_sma_values)}}, ignore_index=True)

# Sort sma_summary_df by 10-year (5040) SMA values in descending order
sma_summary_df.sort_values(by=f'SMA_{sma_windows[1]}', ascending=False, inplace=True)

# Display the DataFrame with 5, 10, and 15-year SMAs for each ticker
print("Top 10 tickers with the best 5, 10, and 15-year SMA values:")
sma_summary_top10 = sma_summary_df.head(10)
print(sma_summary_top10)


/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4019541537.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, **{f'SMA_{window}': value for window, value in zip(sma_windows, latest_sma_values)}}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4019541537.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, **{f'SMA_{window}': value for window, value in zip(sma_windows, latest_sma_values)}}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4019541537.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma

Top 10 tickers with the best 5, 10, and 15-year SMA values:
     Ticker     SMA_1260     SMA_2520     SMA_3780
1   S&P 500  3666.196245  2936.420428  2367.846642
23      CMG  1251.970469   875.005504   657.915900
2      NFLX   394.406190   261.864115   180.101784
30     ADBE   402.642230   257.315695   182.276699
3      COST   378.611199   256.641491   190.005765
40       MA   312.784917   208.336652   148.678400
39       HD   256.650681   182.234892   131.670804
45      FDX   202.055938   181.979656   145.670383
38      MCD   214.881039   158.026850   123.826823
46      MMM   143.145503   138.396439   111.299215


/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4019541537.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, **{f'SMA_{window}': value for window, value in zip(sma_windows, latest_sma_values)}}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4019541537.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma_summary_df.append({'Ticker': ticker, **{f'SMA_{window}': value for window, value in zip(sma_windows, latest_sma_values)}}, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/4019541537.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sma_summary_df = sma

In [11]:
sma_summary_top10.set_index("Ticker", inplace = True)
sma_summary_top10.hvplot(
    xlabel="Ticker", 
    ylabel="SMA Values",
    title='SMA Values (5, 10 and 15 years)',
    width = 1000
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

# Sharpe Ratios

In [12]:
# 5 year Sharp Ratios by stock

# Convert the index to datetime
combined_df.index = pd.to_datetime(combined_df.index)

# Calculate returns over the 5 years
combined_df['Returns_5Y'] = combined_df.groupby('Ticker')['Close'].pct_change(5 * 252)

# Calculate the Sharpe Ratio for each ticker's returns
sharpe_ratios = combined_df.groupby('Ticker').apply(
    lambda group: (group['Returns_5Y'].mean() / group['Returns_5Y'].std()) * np.sqrt(252)
)
sharpe_ratios = sharpe_ratios.rename("Sharpe Ratio (5 years)").reset_index()

# Drop any rows with NaN values 
sharpe_ratios.dropna(subset=['Sharpe Ratio (5 years)'], inplace=True)

# Create a new DataFrame 'sharpe_ratios' to store the results
sharpe_ratios_5y = sharpe_ratios.copy()

# Sory by ascending
sharpe_ratios_5y = sharpe_ratios_5y.sort_values(by='Sharpe Ratio (5 years)', ascending=False)

# Print or further analyze the Sharpe Ratios per ticker
print(sharpe_ratios_5y)


     Ticker  Sharpe Ratio (5 years)
46        V               46.903904
27       MA               42.501201
33      NKE               38.043823
20      HLT               33.589325
29      MCD               29.764749
18    GOOGL               29.573135
35       PG               28.771577
5      AMZN               27.183017
11      CRM               26.916979
10     COST               26.484344
22      JNJ               25.256420
25       KO               24.503655
7        CL               23.236034
44     TSLA               21.383983
21      HSY               20.011142
32     NFLX               19.569861
28      MAR               18.500650
39  S&P 500               18.262450
23      JPM               17.694384
8       CMG               16.768772
2      ADBE               16.679730
0      AAPL               16.322962
19       HD               15.884045
40     SBUX               15.866377
42       SQ               15.445223
30      MMM               15.292952
15      DIS               15

In [13]:
# 10 year Sharp Ratios by stock

# Calculate returns over the 10 years
combined_df['Returns_10Y'] = combined_df.groupby('Ticker')['Close'].pct_change(10 * 252)

# Calculate the Sharpe Ratio for each ticker's returns
sharpe_ratios = combined_df.groupby('Ticker').apply(
    lambda group: (group['Returns_10Y'].mean() / group['Returns_10Y'].std()) * np.sqrt(252)
)
sharpe_ratios = sharpe_ratios.rename("Sharpe Ratio (10 years)").reset_index()

# Drop any rows with NaN values 
sharpe_ratios.dropna(subset=['Sharpe Ratio (10 years)'], inplace=True)

# Create a new DataFrame 'sharpe_ratios' to store the results
sharpe_ratios_10y = sharpe_ratios.copy()

# Sory by ascending
sharpe_ratios_10y = sharpe_ratios_10y.sort_values(by='Sharpe Ratio (10 years)', ascending=False)


# Print or further analyze the Sharpe Ratios per ticker
print(sharpe_ratios_10y)


     Ticker  Sharpe Ratio (10 years)
33      NKE                53.159283
46        V                51.466142
29      MCD                46.336805
27       MA                45.209577
25       KO                44.883485
7        CL                42.795994
35       PG                41.712280
21      HSY                39.191539
44     TSLA                37.791009
18    GOOGL                37.135192
8       CMG                36.706731
22      JNJ                35.474155
30      MMM                32.343555
16      FDX                31.624866
11      CRM                31.175043
39  S&P 500                31.076942
5      AMZN                30.614187
10     COST                29.059849
15      DIS                27.555685
14      DAL                25.490684
32     NFLX                24.817703
40     SBUX                24.405544
28      MAR                23.563108
17       FL                21.817678
23      JPM                21.663675
19       HD                19.664853
0

In [14]:
# 15 year Sharp Ratios by stock

# Calculate returns over the 15 years
combined_df['Returns_15Y'] = combined_df.groupby('Ticker')['Close'].pct_change(15 * 252)

# Calculate the Sharpe Ratio for each ticker's returns
sharpe_ratios = combined_df.groupby('Ticker').apply(
    lambda group: (group['Returns_15Y'].mean() / group['Returns_15Y'].std()) * np.sqrt(252)
)
sharpe_ratios = sharpe_ratios.rename("Sharpe Ratio (15 years)").reset_index()

# Drop any rows with NaN values 
sharpe_ratios.dropna(subset=['Sharpe Ratio (15 years)'], inplace=True)

# Create a new DataFrame 'sharpe_ratios' to store the results
sharpe_ratios_15y = sharpe_ratios.copy()

# Sory by ascending
sharpe_ratios_15y = sharpe_ratios_15y.sort_values(by='Sharpe Ratio (15 years)', ascending=False)

# Print or further analyze the Sharpe Ratios per ticker
print(sharpe_ratios_15y)

     Ticker  Sharpe Ratio (15 years)
7        CL                85.369542
22      JNJ                85.049696
39  S&P 500                83.952682
35       PG                80.906971
33      NKE                59.736918
25       KO                53.118800
40     SBUX                51.971229
26      LUV                49.198032
18    GOOGL                45.069839
46        V                43.647353
15      DIS                43.034705
30      MMM                42.178807
10     COST                41.368983
29      MCD                41.239502
8       CMG                40.881083
6       AXP                40.771243
28      MAR                39.031782
21      HSY                37.633397
23      JPM                35.279237
32     NFLX                33.507245
16      FDX                33.433348
2      ADBE                32.599917
11      CRM                31.985040
43      TGT                30.784372
5      AMZN                30.057227
0      AAPL                28.835964
1

In [15]:
# Merge the DataFrames based on the 'Ticker' column to show 5, 10 and 15 year Sharpe Ratios
sharpe_ratios_combined = sharpe_ratios_5y.merge(sharpe_ratios_10y, on='Ticker')
sharpe_ratios_combined = sharpe_ratios_combined.merge(sharpe_ratios_15y, on='Ticker')

# Rename the Sharpe Ratio columns to specify the timeframes
sharpe_ratios_combined.rename(columns={
    'Sharpe Ratio': 'Sharpe Ratio (5 years)',
    'Sharpe Ratio_x': 'Sharpe Ratio (10 years)',
    'Sharpe Ratio_y': 'Sharpe Ratio (15 years)'
}, inplace=True)

sharpe_ratios_combined.set_index("Ticker", inplace = True)

sharpe_ratios_combined

,Sharpe Ratio (5 years),Sharpe Ratio (10 years),Sharpe Ratio (15 years)
Ticker,,,
V,46.903904,51.466142,43.647353
MA,42.501201,45.209577,23.557603
NKE,38.043823,53.159283,59.736918
MCD,29.764749,46.336805,41.239502
GOOGL,29.573135,37.135192,45.069839
PG,28.771577,41.712280,80.906971
AMZN,27.183017,30.614187,30.057227
CRM,26.916979,31.175043,31.985040
COST,26.484344,29.059849,41.368983


In [16]:
sharpe_ratios_combined.hvplot(
    xlabel="Ticker", 
    ylabel="Sharpe Ratios",
    title='Sharpe Ratios (5, 10 and 15 years)',
    width = 1800
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

# Volume 

In [17]:
# Calculate the total overall volume for each ticker
ticker_total_volume = us_stock_prices_df.groupby('Ticker')['Volume'].sum()

# Sort tickers based on total volume in descending order
sorted_tickers = ticker_total_volume.sort_values(ascending=False)

# Get the top N tickers with the most volume
top_n_tickers = 10  # Change this value to the desired number of top tickers
top_tickers = sorted_tickers.head(top_n_tickers)
top_tickers.columns = ['Ticker', 'Volume']

# Print the list of top tickers with the most volume
print("Top", top_n_tickers, "Tickers with the Most Volume:")
print(top_tickers)


Top 10 Tickers with the Most Volume:
Ticker
AAPL     2.409086e+12
AMZN     7.314642e+11
GOOGL    5.838396e+11
NVDA     3.735403e+11
TSLA     3.210157e+11
MSFT     3.074923e+11
CSCO     2.598988e+11
AMD      1.953171e+11
JPM      1.202160e+11
KO       8.820484e+10
Name: Volume, dtype: float64


In [18]:
top_tickers.hvplot(
    xlabel="Ticker", 
    ylabel='Volume',
    title='Top 10 Tickers with the most Volume',
    width = 1000
).opts(yformatter='%.0f')

:Curve   [Ticker]   (Volume)

In [19]:
#### Find the total volume for 5, 10 and 15 year time frame

# Create a function to calculate total volume for the top N tickers in a given time frame
def calculate_top_tickers_total_volume(data_df, time_frame, top_n_tickers=10):
    # Filter data for the given time frame
    data_df_time_frame = data_df.groupby('Ticker').tail(time_frame)
    
    # Calculate the total volume for each ticker in the time frame
    ticker_total_volume = data_df_time_frame.groupby('Ticker')['Volume'].sum()
    
    # Sort tickers based on total volume in descending order
    sorted_tickers = ticker_total_volume.sort_values(ascending=False)
    
    # Get the top N tickers with the most volume
    top_tickers = sorted_tickers.head(top_n_tickers)
    
    return top_tickers

# Calculate top N tickers' total volume for the specified time frames
top_n_tickers = 10  # Change this value to the desired number of top tickers
time_frames = [1, 5, 10, 15]  # List of time frames (in years)

# Create an empty DataFrame to store the results
v_results_df = pd.DataFrame(columns=['Ticker'] + [f'{years}-Year Volume' for years in time_frames])

# List of unique tickers
tickers = combined_df['Ticker'].unique()

# Loop through each unique ticker
for ticker in tickers:
    ticker_data = combined_df[combined_df['Ticker'] == ticker].copy()
    
    # Calculate total volume for each time frame
    volume_by_time_frame = []
    for time_frame in time_frames:
        top_tickers = calculate_top_tickers_total_volume(ticker_data, time_frame, top_n_tickers)
        total_volume = top_tickers.sum()
        volume_by_time_frame.append(total_volume)
    
    # Create a dictionary to store the results for this ticker
    ticker_results = {'Ticker': ticker}
    for i, years in enumerate(time_frames):
        ticker_results[f'{years}-Year Volume'] = volume_by_time_frame[i]
    
    # Append the results for this ticker to the DataFrame
    v_results_df = v_results_df.append(ticker_results, ignore_index=True)

# Print the DataFrame with results
print(v_results_df)


/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2149202110.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  v_results_df = v_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2149202110.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  v_results_df = v_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2149202110.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  v_results_df = v_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2149202110.py:46: FutureWarning: The frame.append method is deprecated 

     Ticker  1-Year Volume  5-Year Volume  10-Year Volume  15-Year Volume
0      PTON   6.689800e+06   4.118870e+07    1.694141e+08    2.106857e+08
1   S&P 500   3.865960e+09   1.825606e+10    3.535253e+10    5.626857e+10
2      NFLX   6.155900e+06   2.286310e+07    4.864840e+07    7.018570e+07
3      COST   1.714300e+06   7.999500e+06    1.462610e+07    2.298630e+07
4      AMZN   4.063670e+07   2.151494e+08    4.127877e+08    6.355111e+08
5      AAPL   4.528000e+07   2.656249e+08    5.106480e+08    7.747245e+08
6       NKE   6.698700e+06   3.356800e+07    8.790440e+07    1.182807e+08
7       TGT   3.427300e+06   1.920150e+07    4.949630e+07    1.030099e+08
8     GOOGL   1.940310e+07   1.423729e+08    2.680670e+08    4.022401e+08
9      SPOT   1.710300e+06   8.140400e+06    1.494730e+07    2.260810e+07
10       ZM   9.151400e+06   2.415930e+07    5.183420e+07    7.191840e+07
11      DIS   1.274170e+07   7.388440e+07    1.470546e+08    2.106901e+08
12     RBLX   7.976700e+06   3.996820e

/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2149202110.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  v_results_df = v_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2149202110.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  v_results_df = v_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2149202110.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  v_results_df = v_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2149202110.py:46: FutureWarning: The frame.append method is deprecated 

In [20]:
v_results_df.set_index("Ticker", inplace = True)
v_results_df.hvplot(
    xlabel="Ticker", 
    ylabel="Volume",
    title='Volume (5, 10 and 15 years)',
    width = 2500
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

# Annualized and cumulative Returns with S&P 500 as comparison 

## Annualized returns over 5, 10 and 15 years

In [35]:
# Define the time frames in trading days (1, 5, 10, and 15 years)
time_frames = [252, 5 * 252, 10 * 252, 15 * 252]

# Create an empty DataFrame to store the results
a_results_df = pd.DataFrame(columns=['Ticker'] + [f'{years}-Year Annualized Return' for years in time_frames])

# List of unique tickers
tickers = combined_df['Ticker'].unique()

# Loop through each unique ticker
for ticker in tickers:
    ticker_data = combined_df[combined_df['Ticker'] == ticker].copy()
    
    # Calculate annualized returns for each time frame
    returns_by_time_frame = []
    for time_frame in time_frames:
        data_time_frame = ticker_data.tail(time_frame)
        daily_returns = data_time_frame['Close'].pct_change()
        annualized_return = (1 + daily_returns.mean()) ** 252 - 1
        returns_by_time_frame.append(annualized_return)
    
    # Create a dictionary to store the results for this ticker
    ticker_results = {'Ticker': ticker}
    for i, years in enumerate(time_frames):
        column_name = f'{years}-Year Annualized Return'
        ticker_results[column_name] = returns_by_time_frame[i]
    
    # Append the results for this ticker to the DataFrame
    a_results_df = a_results_df.append(ticker_results, ignore_index=True)

# Rename the columns for easier reading
a_results_df.columns = ['Ticker', '1-Year', '5-Year', '10-Year', '15-Year']

# Print the DataFrame with results
print(a_results_df)



/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2651846839.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a_results_df = a_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2651846839.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a_results_df = a_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2651846839.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a_results_df = a_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2651846839.py:29: FutureWarning: The frame.append method is deprecated 

     Ticker    1-Year    5-Year   10-Year   15-Year
0      PTON  0.030938  1.458144  1.458144  1.458144
1   S&P 500  0.195820  0.106480  0.114448  0.112498
2      NFLX  1.206063  0.168742  0.407653  0.557719
3      COST  0.085671  0.237415  0.221441  0.203559
4      AMZN  0.162545  0.138043  0.326647  0.350381
5      AAPL  0.276983  0.356695  0.340718  0.332301
6       NKE  0.019109  0.110553  0.181663  0.200228
7       TGT -0.174314  0.168496  0.151162  0.138143
8     GOOGL  0.337449  0.233324  0.251616  0.227384
9      SPOT  0.712415  0.096401  0.127691  0.127691
10       ZM  0.001388  0.238892  0.238892  0.238892
11      DIS -0.228118 -0.004544  0.077729  0.121657
12     RBLX  0.022392  0.817043  0.817043  0.817043
13      DAL  0.431831  0.058090  0.181428  0.262133
14     MSFT  0.381808  0.320019  0.329283  0.254387
15      LUV -0.098280 -0.049357  0.167926  0.123767
16      AEO  0.754635  0.093211  0.167880  0.163910
17     TSLA  0.141223  1.069723  0.599941  0.597144
18     SBUX 

/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2651846839.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a_results_df = a_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2651846839.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a_results_df = a_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/2651846839.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a_results_df = a_results_df.append(ticker_results, ignore_index=True)


In [36]:
a_results_df.set_index("Ticker", inplace = True)
a_results_df.hvplot(
    xlabel="Ticker", 
    ylabel="Annualized Returns",
    title='Annualized Returns (1, 5, 10 and 15 years)',
    width = 2500
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

## Cumulative Returns over 1, 5, 10 and 15 years

In [31]:
# Define the time frames in trading days (1 yesr, 5, 10, and 15 years)
time_frames = [252, 5 * 252, 10 * 252, 15 * 252]

# Create an empty DataFrame to store the results
c_results_df = pd.DataFrame(columns=['Ticker'] + [f'{years}-Year Cumulative Return' for years in time_frames])

# List of unique tickers
tickers = combined_df['Ticker'].unique()

# Loop through each unique ticker
for ticker in tickers:
    ticker_data = combined_df[combined_df['Ticker'] == ticker].copy()
    
    # Calculate cumulative returns for each time frame
    returns_by_time_frame = []
    for time_frame in time_frames:
        data_time_frame = ticker_data.tail(time_frame)
        cumulative_return = (1 + data_time_frame['Close'].pct_change()).prod() - 1
        returns_by_time_frame.append(cumulative_return)
    
    # Create a dictionary to store the results for this ticker
    ticker_results = {'Ticker': ticker}
    for i, years in enumerate(time_frames):
        column_name = f'{years}-Year Cumulative Return'
        ticker_results[column_name] = returns_by_time_frame[i]
    
    # Append the results for this ticker to the DataFrame
    c_results_df = c_results_df.append(ticker_results, ignore_index=True)

# Rename the columns for easier reading
c_results_df.columns = ['Ticker', '1-Year', '5-Year', '10-Year', '15-Year']

# Print the DataFrame with results
print(c_results_df)



/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/1232710951.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_results_df = c_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/1232710951.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_results_df = c_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/1232710951.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_results_df = c_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/1232710951.py:28: FutureWarning: The frame.append method is deprecated 

     Ticker    1-Year        5-Year       10-Year       15-Year
0      PTON -0.301907  4.111349e-01  4.111349e-01  4.111349e-01
1   S&P 500  0.177884  4.715339e-01  1.524357e+00  2.615953e+00
2      NFLX  0.984344  2.203003e-01  1.006252e+01  9.992417e+01
3      COST  0.057268  1.497115e+00  4.913298e+00  1.011696e+01
4      AMZN  0.076543  3.640316e-01  8.770803e+00  3.191057e+01
5      AAPL  0.224731  2.480591e+00  1.149136e+01  3.501820e+01
6       NKE -0.039279  2.838373e-01  2.564745e+00  6.943935e+00
7       TGT -0.217998  6.063154e-01  1.617630e+00  2.396881e+00
8     GOOGL  0.258878  1.204416e+00  5.406151e+00  1.045124e+01
9      SPOT  0.525761 -1.593499e-01 -9.019076e-03 -9.019076e-03
10       ZM -0.092957  6.511621e-02  6.511621e-02  6.511621e-02
11      DIS -0.269874 -2.606703e-01  4.681706e-01  1.936476e+00
12     RBLX -0.232209  1.097143e-01  1.097143e-01  1.097143e-01
13      DAL  0.352544 -2.552451e-01  1.381042e+00  4.752468e+00
14     MSFT  0.315209  2.134132e+00  1.0

/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/1232710951.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_results_df = c_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/1232710951.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_results_df = c_results_df.append(ticker_results, ignore_index=True)
/var/folders/55/lcmzlzjj1n3gdnlj03vgkdy80000gn/T/ipykernel_76836/1232710951.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_results_df = c_results_df.append(ticker_results, ignore_index=True)


In [32]:
c_results_df.set_index("Ticker", inplace = True)
c_results_df.hvplot(
    xlabel="Ticker", 
    ylabel="Cumulative Returns",
    title='Cumulative Returns (1, 5, 10 and 15 years)',
    width = 2500
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)